In [2]:
import sys, site, pkgutil, subprocess
print("Python:", sys.version)
print("Executable:", sys.executable)
print("User site:", site.getusersitepackages())
print("matplotlib loader:", pkgutil.find_loader("matplotlib"))
print("pip ->", subprocess.check_output([sys.executable, "-m", "pip", "--version"]).decode().strip())


Python: 3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:42:04) [MSC v.1943 64 bit (AMD64)]
Executable: e:\Anaconda\envs\tf\python.exe
User site: C:\Users\Zhongrui Ma\AppData\Roaming\Python\Python310\site-packages
matplotlib loader: None
pip -> pip 25.1 from e:\Anaconda\envs\tf\lib\site-packages\pip (python 3.10)


In [1]:
import matplotlib.pyplot as plt
import time
import warnings
import numpy as np
import os
import scipy.io as sio
import seaborn as sns
import pandas as pd
from operator import truediv
import spectral
from sklearn.manifold import TSNE
from sklearn.decomposition import (IncrementalPCA, PCA)
from sklearn.metrics import (accuracy_score, classification_report,
                             cohen_kappa_score, confusion_matrix)
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.keras.layers import Dense, Dropout, Activation, Reshape, Concatenate
from tensorflow.keras.models import Sequential, Model
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
from keras.optimizers import legacy
from tensorflow.keras.optimizers import Adam
from keras.optimizers import Adam

ModuleNotFoundError: No module named 'matplotlib'